In [296]:
import pandas as pd
!pip install geopandas
!pip install reverse-geocode
import geopandas as gpd


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


## Measurement Type

In [297]:
df_type = pd.read_csv("data/data-1761719326394.csv").rename(columns={"code": "variable_code"})
# df_type.to_csv("output/dt_meas_type.csv", index=False)
df_type

,variable_id,variable_code,name,unit_code,unit_symbol,unit_name,description,created_at,updated_at
0,1,air_humidity,Air humidity,percent,%,NaN,NaN,2025-10-09 15:07:06.75375+00,2025-10-09 15:07:06.75375+00
1,2,air_pressure,Air pressure,hectopascal,hPa,NaN,NaN,2025-10-09 15:07:06.75375+00,2025-10-09 15:07:06.75375+00
2,3,air_temperature,Air temperature,degree Celsius,°C,NaN,NaN,2025-10-09 15:07:06.75375+00,2025-10-09 15:07:06.75375+00
3,4,leaf_wetness,Leaf wetness,minute,min,NaN,NaN,2025-10-09 15:07:06.75375+00,2025-10-09 15:07:06.75375+00
4,5,rainfall,Rainfall,millimetre,mm,NaN,NaN,2025-10-09 15:07:06.75375+00,2025-10-09 15:07:06.75375+00
5,6,soil_humidity,Soil humidity,percent,%,NaN,NaN,2025-10-09 15:07:06.75375+00,2025-10-09 15:07:06.75375+00
6,7,soil_temperature,Soil temperature,degree Celsius,°C,NaN,NaN,2025-10-09 15:07:06.75375+00,2025-10-09 15:07:06.75375+00
7,8,solar_radiation,Solar radiation,watt per square metre,W/m2,NaN,NaN,2025-10-09 15:07:06.75375+00,2025-10-09 15:07:06.75375+00
8,9,wind_direction,Wind direction,degree,°,NaN,NaN,2025-10-09 15:07:06.75375+00,2025-10-09 15:07:06.75375+00
9,10,wind_gusts,Wind gusts,metre per second,m/s,NaN,NaN,2025-10-09 15:07:06.75375+00,2025-10-09 15:07:06.75375+00


## Measurement

In [298]:
df_meas = pd.read_excel("data/data-1761378695532.xlsx").dropna().rename(columns={"station_id": "weather_station"})
df_meas["ts"] = pd.to_datetime(df_meas["ts"])
df_meas["hour"] = df_meas["ts"].dt.strftime("%H")
df_meas["year"] = df_meas["ts"].dt.year
df_meas["month"] = df_meas["ts"].dt.strftime("%Y-%m")   # formatted year-month
df_meas["date"] = df_meas["ts"].dt.strftime("%Y-%m-%d") # formatted date
df_meas = df_meas[["weather_station", "hour", "date", "year", "month", "variable_code", "value", "ts"]]
df_meas = df_meas.pivot_table(
    index=["weather_station", "hour", "date", "year", "month", "ts"],
    columns="variable_code",
    values="value"
).reset_index()
# df_meas = df_meas.drop("ts", axis=1)
df_meas = df_meas.drop_duplicates()
df_meas['date'] = pd.to_datetime(df_meas['date'], format='%Y-%m-%d')
# df_meas.to_csv("output/ft_meas.csv", index=False)
df_meas

variable_code,weather_station,hour,date,year,month,ts,air_humidity,air_pressure,air_temperature,rainfall
0,1325,00,2025-10-11,2025,2025-10,2025-10-11 00:00:00+00:00,88.00,NaN,13.8500,0.0
1,1325,00,2025-10-12,2025,2025-10,2025-10-12 00:00:00+00:00,77.50,NaN,14.5000,0.0
2,1325,00,2025-10-13,2025,2025-10,2025-10-13 00:00:00+00:00,75.50,NaN,14.6500,0.0
3,1325,00,2025-10-14,2025,2025-10,2025-10-14 00:00:00+00:00,100.00,NaN,13.3500,0.0
4,1325,00,2025-10-15,2025,2025-10,2025-10-15 00:00:00+00:00,88.00,NaN,11.7500,0.0
...,...,...,...,...,...,...,...,...,...,...
1510,2382,23,2025-10-18,2025,2025-10,2025-10-18 23:00:00+00:00,67.75,NaN,12.4125,0.0
1511,2382,23,2025-10-19,2025,2025-10,2025-10-19 23:00:00+00:00,68.25,NaN,12.4300,0.0
1512,2382,23,2025-10-20,2025,2025-10,2025-10-20 23:00:00+00:00,87.00,NaN,12.6950,1.4
1513,2382,23,2025-10-21,2025,2025-10,2025-10-21 23:00:00+00:00,91.00,NaN,11.9025,0.0


## Time

In [299]:
dt_date = df_meas[["date", "year", "month"]].drop_duplicates()
dt_date.to_csv("output/dt_date.csv", index=False)
df_meas.drop(["year", "month"], axis=1).to_csv("output/ft_meas.csv", index=False)
dt_date

variable_code,date,year,month
0,2025-10-11,2025,2025-10
1,2025-10-12,2025,2025-10
2,2025-10-13,2025,2025-10
3,2025-10-14,2025,2025-10
4,2025-10-15,2025,2025-10
5,2025-10-16,2025,2025-10
6,2025-10-17,2025,2025-10
7,2025-10-18,2025,2025-10
8,2025-10-19,2025,2025-10
9,2025-10-20,2025,2025-10


## Stazione - Campo - Azienda

In [300]:
import reverse_geocode
import numpy as np

dt_station = pd.read_csv("data/data-1761575739295.csv").rename(columns={"station_id": "weather_station", "name": "name_ws", "latitude": "latitude", "longitude": "longitude", "province": "province"})
dt_station["location"] = ""
dt_station["city"] = dt_station.apply(
    lambda x: reverse_geocode.search([[float(x["latitude"]), float(x["longitude"])]])[0]['city'],
    axis=1
)
dt_station["province"] = dt_station.apply(
    lambda x: reverse_geocode.search([[float(x["latitude"]), float(x["longitude"])]])[0]['county'],
    axis=1
)
dt_station = dt_station[["weather_station", "name_ws", "latitude", "longitude", "location", "city", "province"]]
dt_station.to_csv("output/dt_station.csv", index=False)
dt_station

,weather_station,name_ws,latitude,longitude,location,city,province
0,1325,Macerata Montalbano,43.29223,13.41870,,Villa Potenza,Provincia di Macerata
1,2147,Rolo,44.88542,10.87451,,Rolo,Provincia di Reggio Emilia
2,2262,S Pietro Capofiume,44.65378,11.62264,,San Pietro Capofiume,Bologna
3,2341,Sant'Apollinare Cmt,45.03340,11.82597,,Sant'Apollinare,Provincia di Rovigo
4,2382,Sasso Marconi Arpa,44.43967,11.24125,,Borgonuovo,Bologna


In [301]:
import pandas as pd
import re

# List of CSV files (adjust the path/pattern as needed)
csv_files = [
    "data/TASK 2.3.4 2022-2023 - Flusso Campo Prova.csv",
    "data/TASK 2.3.4 2023-2024 - Flusso Campo Prova.csv"
]

# Empty DataFrame to store appended data
df_campi = pd.DataFrame()

# Function to convert DMS to decimal degrees
def convert(x):
    comp = re.split("°|'|\"|N|E", x)
    return float(comp[0]) + float(comp[1]) / 60 + float(comp[2]) / 3600

# Loop over each CSV file
for file in csv_files:
    df_temp = pd.read_csv(file).dropna().rename(columns={"id-azienda": "id_azienda"})
    df_temp["Coordinate GPS"] = df_temp["Coordinate GPS"].apply(lambda x: x.replace(",", "."))
    df_temp["latitude"] = df_temp["Coordinate GPS"].apply(lambda x: convert(x.split(" ")[0]))
    df_temp["longitude"] = df_temp["Coordinate GPS"].apply(lambda x: convert(x.split(" ")[1]))
    df_campi = pd.concat([df_campi, df_temp], ignore_index=True)

df_campi


,id_azienda,id_campo,Coordinate GPS,latitude,longitude
0,1,1,"44°36'18.5""N 11°33'38.7""E",44.605139,11.560750
1,2,1,"45°00'27.9""N 11°45'08.8""E",45.007750,11.752444
2,3,1,"43°14'06.7""N 13°21'04.5""E",43.235194,13.351250
3,4,1,"44°51'21.2""N 10°48'06.5""E",44.855889,10.801806
4,5,1,"44°27'37.4""N 11°14'39.1""E",44.460389,11.244194
5,6,1,"44°44'58.9""N 12°08'13.1""E",44.749694,12.136972
6,7,2,"44°33'49.2""N 11°32'41.0""E",44.563667,11.544722
7,8,2,"45°01'48.0""N 11°51'26.1""E",45.030000,11.857250
8,9,2,"42°41'39.4""N 11°06'02.6""E",42.694278,11.100722
9,4,2,"44°53'02.3""N 10°47'26.3""E",44.883972,10.790639


In [302]:
# List of CSV files (adjust the path/pattern as needed)
csv_files = [
    "data/TASK 2.3.4 2022-2023 - Flusso Azienda.csv",
    "data/TASK 2.3.4 2023-2024 - Flusso Azienda.csv"
]

# Empty DataFrame to store appended data
df_azienda = pd.DataFrame()

# Loop over each CSV file
for file in csv_files:
    df_temp = pd.read_csv(file).dropna().rename(columns={"id-azienda": "id_azienda"})
    df_azienda = pd.concat([df_azienda, df_temp], ignore_index=True)

df_azienda

,id_azienda,Provincia,Località,Comune
0,1,Bologna,Dugliolo,Budrio
1,2,Rovigo,Arquà Polesine,Arquà Polesine
2,3,Macerata,Rancia,Tolentino
3,4,Reggio Emilia,Fabbrico,Fabbrico
4,5,Bologna,Tizzano,Casalecchio di Reno
5,6,Ferrara,Lagosanto,Lagosanto
6,7,Bologna,Vedrana di budrio,Budrio
7,8,Rovigo,Fenil del turco,Rovigo
8,9,Grosseto,Alberese,Grosseto
9,4,Reggio Emilia,Fabbrico,Fabbrico


In [303]:
gdt_station = gpd.GeoDataFrame(dt_station, geometry=gpd.points_from_xy(dt_station.longitude, dt_station.latitude), crs="EPSG:4326")
gdf_campi = gpd.GeoDataFrame(df_campi, geometry=gpd.points_from_xy(df_campi.longitude, df_campi.latitude), crs="EPSG:4326")

joined = gpd.sjoin_nearest(gdf_campi, gdt_station, how="left", distance_col="distance")

joined = joined.sort_values('distance') #.drop_duplicates('id1')
joined

/usr/local/lib/python3.13/site-packages/geopandas/array.py:408: UserWarning: Geometry is in a geographic CRS. Results from 'sjoin_nearest' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  warnings.warn(


,id_azienda,id_campo,Coordinate GPS,latitude_left,longitude_left,geometry,index_right,weather_station,name_ws,latitude_right,longitude_right,location,city,province,distance
4,5,1,"44°27'37.4""N 11°14'39.1""E",44.460389,11.244194,POINT (11.24419 44.46039),4,2382,Sasso Marconi Arpa,44.43967,11.24125,,Borgonuovo,Bologna,0.020927
7,8,2,"45°01'48.0""N 11°51'26.1""E",45.030000,11.857250,POINT (11.85725 45.03),3,2341,Sant'Apollinare Cmt,45.03340,11.82597,,Sant'Apollinare,Provincia di Rovigo,0.031464
1,2,1,"45°00'27.9""N 11°45'08.8""E",45.007750,11.752444,POINT (11.75244 45.00775),3,2341,Sant'Apollinare Cmt,45.03340,11.82597,,Sant'Apollinare,Provincia di Rovigo,0.077871
3,4,1,"44°51'21.2""N 10°48'06.5""E",44.855889,10.801806,POINT (10.80181 44.85589),1,2147,Rolo,44.88542,10.87451,,Rolo,Provincia di Reggio Emilia,0.078473
0,1,1,"44°36'18.5""N 11°33'38.7""E",44.605139,11.560750,POINT (11.56075 44.60514),2,2262,S Pietro Capofiume,44.65378,11.62264,,San Pietro Capofiume,Bologna,0.078717
9,4,2,"44°53'02.3""N 10°47'26.3""E",44.883972,10.790639,POINT (10.79064 44.88397),1,2147,Rolo,44.88542,10.87451,,Rolo,Provincia di Reggio Emilia,0.083884
2,3,1,"43°14'06.7""N 13°21'04.5""E",43.235194,13.351250,POINT (13.35125 43.23519),0,1325,Macerata Montalbano,43.29223,13.41870,,Villa Potenza,Provincia di Macerata,0.088332
6,7,2,"44°33'49.2""N 11°32'41.0""E",44.563667,11.544722,POINT (11.54472 44.56367),2,2262,S Pietro Capofiume,44.65378,11.62264,,San Pietro Capofiume,Bologna,0.119128
10,10,2,"44°18'19.6""N 11°37'48.5""E",44.305444,11.630139,POINT (11.63014 44.30544),2,2262,S Pietro Capofiume,44.65378,11.62264,,San Pietro Capofiume,Bologna,0.348416
11,6,2,"44°44'59.9""N 12°08'12.9""E",44.749972,12.136917,POINT (12.13692 44.74997),3,2341,Sant'Apollinare Cmt,45.03340,11.82597,,Sant'Apollinare,Provincia di Rovigo,0.420736


In [304]:
joined.columns

Index(['id_azienda', 'id_campo', 'Coordinate GPS', 'latitude_left',
       'longitude_left', 'geometry', 'index_right', 'weather_station',
       'name_ws', 'latitude_right', 'longitude_right', 'location', 'city',
       'province', 'distance'],
      dtype='object')

In [305]:
# List of CSV files (adjust the path/pattern as needed)
csv_files = [
    "data/TASK 2.3.4 2022 - 2023 - Flusso Cultura.csv",
    "data/TASK 2.3.4 2023 - 2024 - Flusso Cultura.csv"
]

# Empty DataFrame to store appended data
df_crop = pd.DataFrame()

# Loop over each CSV file
for file in csv_files:
    df_temp = pd.read_csv(file).dropna().rename(columns={"id-azienda": "id_azienda"}).rename(columns={"Id_azienda": "id_azienda", "Id_campo": "id_campo"})
    df_crop = pd.concat([df_crop, df_temp], ignore_index=True)

df_crop


,id_azienda,id_campo,Specie,Varietà,Intervallo di tempo,da,a
0,1,1,Frumento Duro,Cesare,Ottobre 2022 - luglio 2023,10/1/2022,7/31/2023
1,2,1,Frumento Duro,Cesare,Ottobre 2022 - luglio 2023,10/1/2022,7/31/2023
2,3,1,Frumento Duro,Cesare,Ottobre 2022 - luglio 2023,10/1/2022,7/31/2023
3,4,1,Vite da vino,Lambrusco marani,Marzo 2022 - ottobre 2023,3/1/2022,10/31/2023
4,5,1,Vite da vino,Chardonnay,Marzo 2022 - ottobre 2023,3/1/2022,10/31/2023
5,6,1,Pomodoro da Industria,Fokker,Marzo 2022 - settembre 2023,3/1/2022,9/30/2023
6,7,2,Frumento Duro,Cesare,Ottobre 2023 - luglio 2024,10/1/2023,7/31/2024
7,8,2,Frumento Duro,Cesare,Ottobre 2023 - luglio 2024,10/1/2023,7/31/2024
8,9,2,Frumento Duro,Cesare,Ottobre 2023 - luglio 2024,10/1/2023,7/31/2024
9,4,2,Vite da vino,Lambrusco maestri,Marzo 2023 - ottobre 2024,3/1/2023,10/31/2024


In [ ]:
joined2 = joined[["weather_station", "name_ws", "id_campo", "id_azienda", "latitude_left", "longitude_left", "latitude_right", "longitude_right"]] \
    .merge(df_azienda, on="id_azienda") \
    .merge(df_crop, on=["id_azienda", "id_campo"]) \
    .rename(columns={
        "id_campo": "field",
        "id_azienda": "farm",
        "Località": "location",
        "Comune": "city",
        "Provincia": "province",
        "latitude_right": "latitude_ws",
        "longitude_right": "longitude_ws",
        "latitude_left": "latitude_field",
        "longitude_left": "longitude_field",
        "Specie": "species",
        "Varietà": "variety",
    }).drop_duplicates().reset_index(drop=True)
# dt_station = joined2[["weather_station", "location", "city", "province", "latitude_ws", "longitude_ws"]].drop_duplicates()
# dt_station.to_csv("output/dt_station.csv", index=False)
# dt_station

In [307]:
joined2["field"] = joined2.apply(lambda row: f"f{row['farm']}-f{row['field']}", axis=1)
joined2

,weather_station,name_ws,field,farm,latitude_field,longitude_field,latitude_ws,longitude_ws,province,location,city,species,variety,Intervallo di tempo,da,a
0,2382,Sasso Marconi Arpa,f5-f1,5,44.460389,11.244194,44.43967,11.24125,Bologna,Tizzano,Casalecchio di Reno,Vite da vino,Chardonnay,Marzo 2022 - ottobre 2023,3/1/2022,10/31/2023
1,2341,Sant'Apollinare Cmt,f8-f2,8,45.030000,11.857250,45.03340,11.82597,Rovigo,Fenil del turco,Rovigo,Frumento Duro,Cesare,Ottobre 2023 - luglio 2024,10/1/2023,7/31/2024
2,2341,Sant'Apollinare Cmt,f2-f1,2,45.007750,11.752444,45.03340,11.82597,Rovigo,Arquà Polesine,Arquà Polesine,Frumento Duro,Cesare,Ottobre 2022 - luglio 2023,10/1/2022,7/31/2023
3,2147,Rolo,f4-f1,4,44.855889,10.801806,44.88542,10.87451,Reggio Emilia,Fabbrico,Fabbrico,Vite da vino,Lambrusco marani,Marzo 2022 - ottobre 2023,3/1/2022,10/31/2023
4,2262,S Pietro Capofiume,f1-f1,1,44.605139,11.560750,44.65378,11.62264,Bologna,Dugliolo,Budrio,Frumento Duro,Cesare,Ottobre 2022 - luglio 2023,10/1/2022,7/31/2023
5,2147,Rolo,f4-f2,4,44.883972,10.790639,44.88542,10.87451,Reggio Emilia,Fabbrico,Fabbrico,Vite da vino,Lambrusco maestri,Marzo 2023 - ottobre 2024,3/1/2023,10/31/2024
6,1325,Macerata Montalbano,f3-f1,3,43.235194,13.351250,43.29223,13.41870,Macerata,Rancia,Tolentino,Frumento Duro,Cesare,Ottobre 2022 - luglio 2023,10/1/2022,7/31/2023
7,2262,S Pietro Capofiume,f7-f2,7,44.563667,11.544722,44.65378,11.62264,Bologna,Vedrana di budrio,Budrio,Frumento Duro,Cesare,Ottobre 2023 - luglio 2024,10/1/2023,7/31/2024
8,2262,S Pietro Capofiume,f10-f2,10,44.305444,11.630139,44.65378,11.62264,Bologna,Casalfiumanese,Casalfiumanese,Vite da vino,Trebbiano,Marzo 2023 - ottobre 2024,3/1/2023,10/31/2024
9,2341,Sant'Apollinare Cmt,f6-f2,6,44.749972,12.136917,45.03340,11.82597,Ferrara,Lagosanto,Lagosanto,Pomodoro da Industria,Fokker,Marzo 2023 - settembre 2024,3/1/2023,9/30/2024


In [308]:
dt_field = joined2.drop(["Intervallo di tempo"], axis=1).drop_duplicates().rename(columns={"da": "valid_from", "a": "valid_to"})
dt_field.to_csv("output/dt_field.csv", index=False)
dt_field

,weather_station,name_ws,field,farm,latitude_field,longitude_field,latitude_ws,longitude_ws,province,location,city,species,variety,valid_from,valid_to
0,2382,Sasso Marconi Arpa,f5-f1,5,44.460389,11.244194,44.43967,11.24125,Bologna,Tizzano,Casalecchio di Reno,Vite da vino,Chardonnay,3/1/2022,10/31/2023
1,2341,Sant'Apollinare Cmt,f8-f2,8,45.030000,11.857250,45.03340,11.82597,Rovigo,Fenil del turco,Rovigo,Frumento Duro,Cesare,10/1/2023,7/31/2024
2,2341,Sant'Apollinare Cmt,f2-f1,2,45.007750,11.752444,45.03340,11.82597,Rovigo,Arquà Polesine,Arquà Polesine,Frumento Duro,Cesare,10/1/2022,7/31/2023
3,2147,Rolo,f4-f1,4,44.855889,10.801806,44.88542,10.87451,Reggio Emilia,Fabbrico,Fabbrico,Vite da vino,Lambrusco marani,3/1/2022,10/31/2023
4,2262,S Pietro Capofiume,f1-f1,1,44.605139,11.560750,44.65378,11.62264,Bologna,Dugliolo,Budrio,Frumento Duro,Cesare,10/1/2022,7/31/2023
5,2147,Rolo,f4-f2,4,44.883972,10.790639,44.88542,10.87451,Reggio Emilia,Fabbrico,Fabbrico,Vite da vino,Lambrusco maestri,3/1/2023,10/31/2024
6,1325,Macerata Montalbano,f3-f1,3,43.235194,13.351250,43.29223,13.41870,Macerata,Rancia,Tolentino,Frumento Duro,Cesare,10/1/2022,7/31/2023
7,2262,S Pietro Capofiume,f7-f2,7,44.563667,11.544722,44.65378,11.62264,Bologna,Vedrana di budrio,Budrio,Frumento Duro,Cesare,10/1/2023,7/31/2024
8,2262,S Pietro Capofiume,f10-f2,10,44.305444,11.630139,44.65378,11.62264,Bologna,Casalfiumanese,Casalfiumanese,Vite da vino,Trebbiano,3/1/2023,10/31/2024
9,2341,Sant'Apollinare Cmt,f6-f2,6,44.749972,12.136917,45.03340,11.82597,Ferrara,Lagosanto,Lagosanto,Pomodoro da Industria,Fokker,3/1/2023,9/30/2024


In [309]:
# Convert to datetime
dt_field['valid_from'] = pd.to_datetime(dt_field['valid_from'], format='%m/%d/%Y')
dt_field['valid_to'] = pd.to_datetime(dt_field['valid_to'], format='%m/%d/%Y')

# Create a date range for each row
dt_field['date'] = dt_field.apply(lambda row: pd.date_range(row['valid_from'], row['valid_to']), axis=1)

# Expand rows
ft_colture = dt_field[['field', 'date']].explode('date').reset_index(drop=True)
dt_field = dt_field.drop(['date'], axis=1)
ft_colture.drop_duplicates().to_csv("output/ft_colture.csv", index=False)
ft_colture



,field,date
0,f5-f1,2022-03-01
1,f5-f1,2022-03-02
2,f5-f1,2022-03-03
3,f5-f1,2022-03-04
4,f5-f1,2022-03-05
...,...,...
5423,f9-f2,2024-07-27
5424,f9-f2,2024-07-28
5425,f9-f2,2024-07-29
5426,f9-f2,2024-07-30


In [ ]:
# This is the corrected query! But the dates of measurements and colture do not match yet.
# da_field_meas = ft_colture \
#     .merge(dt_field, on=["field"]) \
#     .merge(df_meas, on=["weather_station", "date"]) \
#     .rename(columns={"date_y": "date"}) \
#     .drop("date_x", axis=1)

da_field_meas = dt_field.merge(df_meas, on=["weather_station"])
da_field_meas = da_field_meas[["field", "hour", "date", "ts", "air_temperature", "air_humidity", "air_pressure", "rainfall"]]
da_field_meas.to_csv("output/ft_field_meas.csv", index=False)
da_field_meas


,field,hour,date,ts,air_temperature,air_humidity,air_pressure,rainfall
0,f5-f1,00,2025-10-11,2025-10-11 00:00:00+00:00,14.7175,74.50,NaN,0.0
1,f5-f1,00,2025-10-12,2025-10-12 00:00:00+00:00,16.0025,76.00,NaN,0.0
2,f5-f1,00,2025-10-13,2025-10-13 00:00:00+00:00,14.8625,82.50,NaN,0.0
3,f5-f1,00,2025-10-14,2025-10-14 00:00:00+00:00,15.2800,80.00,NaN,0.0
4,f5-f1,00,2025-10-15,2025-10-15 00:00:00+00:00,15.0475,73.75,NaN,0.0
...,...,...,...,...,...,...,...,...
3637,f9-f2,23,2025-10-18,2025-10-18 23:00:00+00:00,12.4125,67.75,NaN,0.0
3638,f9-f2,23,2025-10-19,2025-10-19 23:00:00+00:00,12.4300,68.25,NaN,0.0
3639,f9-f2,23,2025-10-20,2025-10-20 23:00:00+00:00,12.6950,87.00,NaN,1.4
3640,f9-f2,23,2025-10-21,2025-10-21 23:00:00+00:00,11.9025,91.00,NaN,0.0
